# Install necessary libraries


In [ ]:
!pip install torch torchvision torchmetrics thop


In [ ]:
! pip install torchprofile

In [ ]:
!pip install -U fvcore

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
import os


from PIL import Image

#from torchmetrics import JaccardIndex
#from thop import profile, clever_format
import time

#from fvcore.nn import FlopCountAnalysis, flop_count_table
#import torchprofile

#from models.deeplabv2.deeplabv2 import get_deeplab_v2


from torchmetrics import JaccardIndex
from thop import profile, clever_format
import time

from fvcore.nn import FlopCountAnalysis, flop_count_table
import torchprofile





In [ ]:
from google.colab import drive

drive.mount('/content/Drive')
get_ipython().system('/content/Drive/MyDrive/Cityscapes/Cityspaces')

In [ ]:

class CityscapesDataset(Dataset):
  def __init__(self, root_dir, im_transform ):

    """
    Args:
    root_dir (string): Directory with all the images.
    transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.root_dir = root_dir
    self.im_transform = im_transform
    #self.lab_transform = lab_transform
    self.images = []
    for subdir, dirs, files in os.walk(root_dir):
      for file in files:
        if file.endswith('_gtFine_color.png'):
          self.images.append(os.path.join(subdir, file))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = self.images[idx]
    image = Image.open(img_name).convert('RGB')
    label_name = img_name.replace('_gtFine_color.png', '_gtFine_labelTrainIds.png')  #labelTrainIds
    label = Image.open(label_name)


    # Resize label using nearest-neighbor interpolation
    label = TF.resize(label, (1024, 512), interpolation=transforms.InterpolationMode.NEAREST)
    label = np.array(label)  # Convert to numpy array
    label = torch.from_numpy(label).long()  # Convert to LongTensor


    if self.im_transform:

      image = self.im_transform(image)

    # if self.lab_transform:
    #   label = self.lab_transform(label)

    return image, label



**Model Clone**

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/Gabrysse/MLDL2024_project1.git


In [ ]:
# Navigate to the project directory
%cd MLDL2024_project1

In [ ]:
from models.deeplabv2.deeplabv2 import get_deeplab_v2

In [ ]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


**Parameters**

In [ ]:
# Define training parameters
epochs = 50
learning_rate =  0.0001
batch_size = 4
train_resolution = (1024, 512)
test_resolution = (1024, 512)

#learning_rate = 0.0001 batch_size = 4

**Train Loader**

In [ ]:
# Define a transform if you need to preprocess the images
transformed_dataset = CityscapesDataset(root_dir='/content/Drive/MyDrive/Cityscapes/Cityspaces/gtFine/train',
im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(train_resolution),

]), )


train_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




In [ ]:
# Load the DeepLabV2 model
model = get_deeplab_v2(num_classes=19, pretrain=False)
model = model.to(device)



# Define the CrossEntropyLoss with ignore_index set to 255
criterion = nn.CrossEntropyLoss(ignore_index=255)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Define the metric for mIoU
miou_metric = JaccardIndex(num_classes=19, task='multiclass' , ignore_index=255).to(device)


In [ ]:
# Function to compute latency
def measure_latency(model, input_tensor, repetitions=100):
    model.eval()
    with torch.no_grad():
        start = time.time()
        for _ in range(repetitions):
            _ = model(input_tensor)
        end = time.time()
    latency = (end - start) / repetitions
    return latency



# Measure FLOPs and number of parameters


dummy_input = torch.randn(1, 3, 1024, 512).to(device)

height = 1024
width = 512
image =torch.zeros((1,3, height, width)).to(device)   # torch.randn(1,3, 1024, 512).to(device)#

model.eval()
flops = FlopCountAnalysis(model, image)
print(flop_count_table(flops))



| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 0.375T     |
|  conv1                         |  9.408K                |  1.233G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  16.777M   |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  7.155G    |
|   layer1.0                     |   75.008K              |   2.487G   |
|    layer1.0.conv1              |    4.096K              |    0.136G  |
|    layer1.0.bn1                |    0.128K              |    4.244M  |
|    layer1.0.conv2              |    36.864K             |    1.222G  |
|    layer1.0.bn2                |    0.128K       

In [ ]:
# Measure FLOPs and parameters using torchprofile
dummy_input = torch.randn(1, 3, 1024,512).to(device)
model.eval()
flops = torchprofile.profile_macs(model, args=(dummy_input,))

params = sum(p.numel() for p in model.parameters())

print(f' flops={flops}\n params={params} ')


 flops=374180359552
 params=43901068 


**Training**

In [ ]:
from google.colab import drive
from pathlib import Path
import os

drive.mount('/content/Drive')
get_ipython().system('/content/Drive/MyDrive/Checkpoints/deeplap/lr0001b4')


In [ ]:
# Function to save the model
def save_checkpoint(epoch, model, optimizer, save_dir='/content/Drive/MyDrive/Checkpoints/deeplap/lr0001b4'):
    # Ensure the save directory exists
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    # Define the model filename with the epoch number
    checkpoint_filename = f'checkpoint_epoch_{epoch}.pth'
    checkpoint_path = os.path.join(save_dir, checkpoint_filename)

    # Save the model and optimizer state dictionaries
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f'Model and optimizer saved to {checkpoint_path}')

# Function to load the model
def load_checkpoint(epoch, model, optimizer, save_dir='/content/Drive/MyDrive/Checkpoints/deeplap/lr0001b4'):
    checkpoint_filename = f'checkpoint_epoch_{epoch}.pth'
    checkpoint_path = os.path.join(save_dir, checkpoint_filename)

    # Check if the file exists
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"The specified file was not found: {checkpoint_path}")

    # Load the model and optimizer state dictionaries
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']

    print(f'Model and optimizer loaded from {checkpoint_path}, resuming at epoch {start_epoch}')
    return model, optimizer, start_epoch

In [ ]:
# Training loop

# Resume training from the last checkpoint if available
resume_training =True  # Set this to True if you want to resume training

epochs = 50 # Set this to the total number of epochs you want to train
if resume_training:
    epoch_to_resume =24 #9,19,29,39,49  # Set this to the epoch from which you want to resume
    try:
        model, optimizer, start_epoch = load_checkpoint(epoch_to_resume, model, optimizer)

    except FileNotFoundError as e:
        print(e)
        start_epoch = 0
else:
    start_epoch = 0

# Training loop
for epoch in range(start_epoch, epochs):
#for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    miou_metric.reset()
    counter=1
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        labels = labels.squeeze(1)

        loss = criterion(outputs[0], labels.long())
        #print(loss)
        loss.backward()
        optimizer.step()
       # print(counter)
        counter+=1

        running_loss += loss.item()

        miou_metric.update(outputs[0].argmax(dim=1), labels)


         # Save the model every 10 epochs
    if (epoch+1) % 5 == 0:
        save_checkpoint(epoch, model, optimizer)


    miou = miou_metric.compute().item()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}, Train mIoU: {miou}')

# Measure latency after training
latency = measure_latency(model, dummy_input)
print(f"FLOPs: {flops}, Params: {params}, Latency: {latency:.6f} seconds")

print("Training completed!")


**Test Loader**

In [ ]:
# Define a transform if you need to preprocess the images
transformed_dataset = CityscapesDataset(root_dir='/content/Drive/MyDrive/Cityscapes/Cityspaces/gtFine/val', #/content/Cityscapes/Cityspaces/gtFine/train
im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(test_resolution),

]), )


test_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




**Inference and mIoU calculation on test set**

In [ ]:



model.eval()
miou_metric.reset()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        # Assuming outputs is of shape (N, C, H, W) and labels is of shape (N, H, W)
        # If outputs is already (N, C, H, W) and labels is (N, H, W), you should not squeeze labels
         #labels = labels.squeeze(0)  # Remove this line if labels are already of shape (N, H, W)

        # Get predictions
        preds = outputs.argmax(dim=1)  # Now preds will be of shape (N, H, W)

        miou_metric.update(preds, labels)

miou = miou_metric.compute().item()
print(f'Test mIoU: {miou}')


